In [1]:
import pyspark
import findspark

In [2]:
print(findspark.find())
findspark.init("/opt/spark")

/opt/spark


In [3]:
from pyspark.sql import SparkSession

sc = SparkSession \
    .builder \
    .appName("cluster") \
    .master('local[2]') \
    .getOrCreate()

22/02/03 17:07:05 WARN Utils: Your hostname, CT-LP-234 resolves to a loopback address: 127.0.1.1; using 172.30.150.107 instead (on interface eth0)
22/02/03 17:07:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/02/03 17:07:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
training = sc.createDataFrame([
     (0, 'i like apple pie for dessert', 1.0),
     (1, 'i dont drive fast cars', 0.0),
     (2, 'data science is fun', 1.0),
     (3, 'chocolate is not my favorite', 0.0),
     (4, 'my favorite movie is predator', 1.0)],
     ['id', 'text', 'label'])

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [7]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01, featuresCol='features',labelCol='label')

In [8]:
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [9]:
import mlflow

In [10]:
with mlflow.start_run() as run1:
    mlflow.pyspark.ml.autolog()
    model = pipeline.fit(training)

22/02/03 17:11:17 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/02/03 17:11:17 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [14]:
mlflow.end_run()

In [15]:
sc.stop()